## Functions for loading and plotting SD and DALEC data from multiple days

I've already prototyped some of this in `SD_8-20Aug22.ipynb`. So just gonna generalise into useful functions

In [4]:
# probably won't need all of this stuff, but why not import it all?

import SD_raster_loading
import SD_NC_loading
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dalecLoad
import spectralConv
import os

In [17]:
# now gonna try to look at all SD files:

def load_multiple_SDs(SD_directory, coord, pixel_grid_shape=(1, 1)):
    '''
    Loads all L2R netcdfs in a given directory. Then extracts a grid of shape=pixel_grid_shape at the given coord
    Returns a pandas DF with Date, Wavelength, and rho_s columns
    Note that the rho_s columns will be formatted as per `SD_NC_loading.get_SD_NC_Spectra_grid()`
    '''
    # could use list comprehensions in a few places here if things start getting slow!
    SD_files = []
    for file in os.listdir(SD_directory):
        if file.endswith("L2R.nc"):
            SD_files.append(os.path.join(SD_directory, file))


    ncdf_dates = []
    indexes = []
    SD_spect_list = []

    for i in range(len(SD_files)):
        f = netCDF4.Dataset(SD_files[i])
        SD_spect = SD_NC_loading.get_SD_NC_Spectra_grid(f, coord[0], coord[1], shape=(3, 3))
        ncdf_dates.append(f.isodate)
        indexes.append(i)
        SD_spect_list.append(SD_spect)
    
    
    # currently my code which removes images from the same date relies on the images being sorted in date order ...
    SD_spect_list_sorted = [x for _, x in sorted(zip(ncdf_dates, SD_spect_list))]
    sorted_dates = sorted(ncdf_dates)
    
    SD_df = None # this is almost definitely uneccesary
    
    # perhaps have removing images from same date as an option??
    for i in range(len(SD_spect_list_sorted)):
        SD_spect = SD_spect_list_sorted[i]
        date = sorted_dates[i]
        if date[:10] != sorted_dates[i-1][:10]: # can't be bothered to deal with multiple images from same day right now
            SD_df_tmp = SD_spect.copy()
            SD_df_tmp['Date'] = pd.to_datetime(date)
            SD_df_tmp['Date'] = SD_df_tmp['Date'].dt.date # just removes the time aspect from the variable
            SD_df_tmp.set_index(['Date', 'Wavelength'], inplace=True)
            if SD_df is None:
                SD_df = SD_df_tmp.copy()
            else:
                SD_df = pd.concat([SD_df, SD_df_tmp])
        else:
            print('...skipping duplicate date entry')
    # don't really need to sort, but in case I change something its good to have
    return SD_df.sort_values(['Date', 'Wavelength']) 

In [26]:
SD_directory = ('C:/Users/daa5/' +
                'OneDrive - University of Stirling/' +
                'PlanetData/' +
                'Airthrey_8-20Aug-TOAR_psscene_analytic_8b_udm2/' +
                'acolite_output/')

coord = [56.14693897799395, -3.923458784671348] # this is approx the location of the DALEC


SD_df = load_multiple_SDs(SD_directory, coord)
SD_df

...skipping duplicate date entry
...skipping duplicate date entry


rho_s_49_86  rho_s_49_87  rho_s_49_88   rho_s_50_86  \
Date       Wavelength                                                        
2022-08-10 444.0       0.014849082  0.017234027  0.013689813    0.01235939   
           492.0       0.013610188  0.013732793  0.014223221   0.011954372   
           533.0       0.024336988  0.023728922  0.022576505    0.02337684   
           566.0        0.02928955  0.026712634  0.025994726   0.028898228   
           612.0       0.021671271  0.020159846  0.020778213     0.0168949   
           666.0       0.019844772   0.02232088  0.020670263    0.01711617   
           707.0        0.04264811  0.041237798  0.038301744    0.03692855   
           866.0        0.06340983   0.06696183   0.07192138    0.05546401   
2022-08-11 444.0       0.012887616  0.012312349  0.012752273  0.0104163885   
           492.0       0.016868625   0.01659519   0.01553158   0.014984443   
           533.0         0.0272826  0.026647702  0.025441097   0.022709006   
           566.0       0.031769257  0.030765718  0.029956251   0.029146638   
           612.0        0.02824181  0.025958505   0.02360602    0.02445849   
           666.0       0.025744613  0.023392996  0.022715855   0.021325698   
           707.0       0.048778728   0.04730209   0.04605238    0.04321129   
           866.0        0.11999861   0.11415913   0.11498619   0.099518225   
2022-08-13 444.0       0.012737081  0.013761592  0.013359167    0.01690484   
           492.0       0.013855458  0.016052853  0.013855458    0.01339602   
           533.0       0.024998838  0.020866094  0.020968638   0.026806898   
           566.0       0.034623317  0.034415632   0.03528085   0.032234054   
           612.0       0.026796646  0.025849039   0.02843601   0.027379636   
           666.0       0.021108413  0.019050065  0.018325506   0.017944096   
           707.0       0.047138497  0.044681527  0.042263057   0.043674044   
           866.0        0.10927225   0.09460875  0.093133755   0.086954296   
2022-08-17 444.0       0.014341316  0.016170692   0.01595554   0.018714838   
           492.0       0.018310944  0.016831134   0.01647714   0.016831134   
           533.0       0.033005953  0.033541422  0.033507958   0.032604277   
           566.0       0.043382205   0.04399426   0.04382426    0.04263397   
           612.0       0.028758785   0.03209416  0.032846883   0.028507601   
           666.0        0.02236466  0.023302097  0.023939429   0.021276953   
           707.0       0.052944496  0.050326142   0.04830174    0.04929422   
           866.0        0.06865051   0.06913929   0.07223377   0.059517562   
2022-08-20 444.0       0.011257653  0.016698709  0.011628192   0.009181508   
           492.0       0.016946774  0.017212091  0.020990163   0.014889763   
           533.0        0.03456989    0.0357092    0.0341555   0.034431767   
           566.0       0.040657233  0.044314962  0.045615423    0.04104427   
           612.0       0.025645854  0.025868488  0.026833111   0.024829421   
           666.0       0.023573374  0.023650728   0.02411484   0.022490267   
           707.0        0.06256248  0.059902243  0.059370045    0.06231698   
           866.0        0.10866624   0.09695275   0.09008233    0.08258934   

                       rho_s_50_87  rho_s_50_88   rho_s_51_86   rho_s_51_87  \
Date       Wavelength                                                         
2022-08-10 444.0       0.015292178  0.014030832  0.0114720315   0.011847491   
           492.0       0.012537072  0.012107725   0.011647631   0.011893033   
           533.0       0.022384407  0.021423724   0.021872072   0.021455752   
           566.0       0.027299918  0.025407244   0.028311161   0.027365169   
           612.0       0.016929273  0.018407362   0.018407362   0.018029293   
           666.0       0.017331634  0.015966881   0.015751362   0.014601784   
           707.0         0.0368904  0.033723194   0.030363355   0.032196227   
           866.0        0.05813098 

In [32]:
def load_SD_summarise_multiple_DALEC_days(DALEC_directory, RSR_doves_file='non-DALEC-data/RSR-Superdove.csv',
                                          file_names=None, dalec_summary_function=dalecLoad.uniform_grid_spectra_mean,
                                          DALEC_col_name='DALEC_mean_Rrs'):
    '''
    Loads multiple DALEC log files and then carries out the specified daily summary operation on these.
    Then resamples to superdoves wavebands and saves the data in a nice dataframe with Date, Wavelength and 
    '''
    supported_functions = [dalecLoad.uniform_grid_spectra_mean]
    if dalec_summary_function not in supported_functions:
        print("I've not tested this summary function yet! It might not (probably won't) work! ")
    
    RSR_doves = pd.read_csv(RSR_doves_file)
    
    if file_names is None: # if None, then load all DALEC transect (.dtf) files in the directory
        DALEC_files = []
        for file in os.listdir(DALEC_directory):
            if file.endswith(".dtf"):
                DALEC_files.append(os.path.join(DALEC_directory, file))
    else:
        DALEC_files = [DALEC_directory + file for file in file_names]

    DALEC_df = None # just initialise as None idk
    
    # assuming that the spectral wavelength info is the same for each file
    # this is almost definitely always the case... (unless perhaps we used a different DALEC?)
    spect_wavelengths = dalecLoad.load_DALEC_spect_wavelengths(DALEC_files[0])
    # would be a bit nicer to not hard code these, but would require also reading a SD file
    # see SD_NC_loading.get_SD_NC_Spectra() for how to do this
    doves_wavelengths = [444., 492., 533., 566., 612., 666., 707., 866.]
    
    
    
    # as we know, loading DALEC files isnae that fast...
    # currently some weird stuff will happen if we include log files which are from serial output
    # basically need dalecLoad.load_DALEC_log() to be super robust for this to work! 
    for file in DALEC_files:
        print('loading ... ' + str(file))
        dalec_log = dalecLoad.load_DALEC_log(file)
        mean_spect = dalec_summary_function(dalec_log, spect_wavelengths)
        DALEC_SD = spectralConv.SD_band_calc(RSR_doves, mean_spect['Rrs_mean'].values,
                                             RSR_doves['Wavelength (nm)'].values)
        DALEC_df_tmp = pd.DataFrame(data=DALEC_SD, columns=[DALEC_col_name])
        DALEC_df_tmp['Date'] = pd.to_datetime(dalec_log[' UTC Date'].iloc[0])
        DALEC_df_tmp['Date'] = DALEC_df_tmp['Date'].dt.date # just removes the time aspect from the variable
        DALEC_df_tmp['Wavelength'] = doves_wavelengths
        DALEC_df_tmp.set_index(['Date', 'Wavelength'], inplace=True)
        if DALEC_df is None:
            DALEC_df = DALEC_df_tmp.copy()
        else:
            DALEC_df = pd.concat([DALEC_df, DALEC_df_tmp])

    return DALEC_df





In [34]:
DALEC_df = load_SD_summarise_multiple_DALEC_days('data/log82_89/')
DALEC_df

loading ... data/log82_89/LOG_0082.dtf
loading ... data/log82_89/LOG_0083.dtf
loading ... data/log82_89/LOG_0084.dtf
loading ... data/log82_89/LOG_0085.dtf
loading ... data/log82_89/LOG_0086.dtf
loading ... data/log82_89/LOG_0087.dtf
loading ... data/log82_89/LOG_0088.dtf
loading ... data/log82_89/LOG_0089.dtf


DALEC_mean_Rrs
Date       Wavelength                
2022-08-08 444.0             0.001780
           492.0             0.002995
           533.0             0.005177
           566.0             0.006222
           612.0             0.004774
...                               ...
2022-08-15 566.0             0.012465
           612.0             0.008536
           666.0             0.007134
           707.0             0.011419
           866.0             0.001412

[64 rows x 1 columns]

In [35]:
# join the dfs together and remove NAs so that we only get overlapping dates
superDuperDF = DALEC_df.join(SD_df, on=['Date', 'Wavelength']).dropna() 
superDuperDF

DALEC_mean_Rrs  rho_s_49_86  rho_s_49_87  rho_s_49_88  \
Date       Wavelength                                                          
2022-08-10 444.0             0.002926  0.014849082  0.017234027  0.013689813   
           492.0             0.003997  0.013610188  0.013732793  0.014223221   
           533.0             0.006104  0.024336988  0.023728922  0.022576505   
           566.0             0.007177   0.02928955  0.026712634  0.025994726   
           612.0             0.005585  0.021671271  0.020159846  0.020778213   
           666.0             0.004232  0.019844772   0.02232088  0.020670263   
           707.0             0.006749   0.04264811  0.041237798  0.038301744   
           866.0             0.001269   0.06340983   0.06696183   0.07192138   
2022-08-11 444.0             0.002647  0.012887616  0.012312349  0.012752273   
           492.0             0.003834  0.016868625   0.01659519   0.01553158   
           533.0             0.006004    0.0272826  0.026647702  0.025441097   
           566.0             0.007173  0.031769257  0.030765718  0.029956251   
           612.0             0.005457   0.02824181  0.025958505   0.02360602   
           666.0             0.004192  0.025744613  0.023392996  0.022715855   
           707.0             0.006709  0.048778728   0.04730209   0.04605238   
           866.0             0.001232   0.11999861   0.11415913   0.11498619   
2022-08-13 444.0             0.003807  0.012737081  0.013761592  0.013359167   
           492.0             0.006062  0.013855458  0.016052853  0.013855458   
           533.0             0.010203  0.024998838  0.020866094  0.020968638   
           566.0             0.012801  0.034623317  0.034415632   0.03528085   
           612.0             0.008585  0.026796646  0.025849039   0.02843601   
           666.0             0.007026  0.021108413  0.019050065  0.018325506   
           707.0             0.011576  0.047138497  0.044681527  0.042263057   
           866.0             0.002524   0.10927225   0.09460875  0.093133755   

                        rho_s_50_86  rho_s_50_87  rho_s_50_88   rho_s_51_86  \
Date       Wavelength                                                         
2022-08-10 444.0         0.01235939  0.015292178  0.014030832  0.0114720315   
           492.0        0.011954372  0.012537072  0.012107725   0.011647631   
           533.0         0.02337684  0.022384407  0.021423724   0.021872072   
           566.0        0.028898228  0.027299918  0.025407244   0.028311161   
           612.0          0.0168949  0.016929273  0.018407362   0.018407362   
           666.0         0.01711617  0.017331634  0.015966881   0.015751362   
           707.0         0.03692855    0.0368904  0.033723194   0.030363355   
           866.0         0.05546401   0.05813098  0.052272867   0.057346676   
2022-08-11 444.0       0.0104163885  0.010788912  0.010890513   0.008485096   
           492.0        0.014984443  0.013859486  0.013160004   0.014102763   
           533.0        0.022709006  0.022995008  0.024551788   0.021787282   
           566.0        0.029146638  0.026976151   0.02499914   0.026490081   
           612.0         0.02445849  0.022855738  0.021695996    0.01927337   
           666.0        0.021325698  0.019899582  0.019756949   0.019899582   
           707.0         0.04321129  0.042074524  0.043628056    0.03468102   
           866.0        0.099518225    0.0910748   0.08729085    0.08371282   
2022-08-13 444.0         0.01690484  0.015480062  0.011821886   0.007349544   
           492.0         0.01339602  0.013921076  0.013428836   0.013002143   
           533.0        0.026806898    0.0235651  0.022950396   0.024043096   
           566.0        0.032234054  0.032060847  0.033688646     0.0332385   
           612.0        0.027379636  0.029018685  0.029528417   0.026650878   
           666.0        0.017944096  0.017219301  0.017219301   0.015463824   
           707.0        0.043674044   0.040609

In [ ]:
# need to finish this! 

def multiDaySpectraPlot(superDuperDF, )

fig, ax = plt.subplots(1, 3, figsize=(25, 7))

for i, date in zip(range(3), superDuperDF.index.get_level_values(0).unique()):
    x = superDuperDF.index.get_level_values(1).unique()
    y = superDuperDF.loc[[date]]['DALEC_mean_Rrs'].values
    ax[i].plot(x, y, label='DALEC Mean')
    for col in list(superDuperDF.columns.values)[1:]:
        y = superDuperDF.loc[[date]][col].values/np.pi
        ax[i].plot(x, y,
                   color='red',
                   label='SuperDoves @ DALEC loc.',
                   marker='o',
                   alpha=0.2)
    ax[i].set_ylim([0, 0.03])
    ax[i].set_title(str(date))
    
    ax[i].set_xlabel('Wavelength (nm)')
    ax[i].set_ylabel('$R_{rs}$ $(sr^{-1}$)')


    # this code removes duplicate labels in the legend. Naughty
    handles, labels = ax[i].get_legend_handles_labels()
    newLabels, newHandles = [], []
    for handle, label in zip(handles, labels):
        if label not in newLabels:
            newLabels.append(label)
            newHandles.append(handle)
    ax[i].legend(newHandles, newLabels)

    ax[i].grid()
plt.show()